In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
# fix seeds, make everything reproducible, etc (at least try to).
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DATADIR = "../data"

MAX_WORDS = 4

In [4]:
# !pip install aicrowd-cli
%load_ext aicrowd.magic

In [5]:
%aicrowd login

API Key valid
Gitlab oauth token invalid or absent.
It is highly recommended to simply run `aicrowd login` without passing the API Key.
Saved details successfully!


In [6]:
import re
if Path(DATADIR).exists():
  !rm -rf $DATADIR
!mkdir $DATADIR
%aicrowd ds dl -c htrec-2022 -o $DATADIR

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

In [7]:
import pywer
train_df = pd.read_csv( f"{DATADIR}/train.csv")
test_df = pd.read_csv(f"{DATADIR}/test.csv")

word_regex = re.compile("\W+")
word_regex2 = re.compile("(\W+)")

train_df.head()

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM
0,ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει,ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,1
1,τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχ...,του β ου του καλεαυτοὺς πολλαγινεσθαι συγχωρ όν,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,2
2,τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ,τες εμπυριζου σιμαμπελῶνα ἀλλακαι ὅδξα,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,3
3,τῆς ἡδεῖας πλεονεξίας πολλοὺς εἰς τὴν τῶν ἀλλ,της ἐδίας πλσον ἐξιας πολλους ἐις τὴν τῶν ἀλ,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,4
4,οτρίων ἐπιθυμίαν προκαλούμενος ἐμπυρί,λοτρλων ἐπιθυμιαν προκαλουμένος ἐμπυρι,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,5


In [8]:
# !pip install nltk

In [9]:
for lib_dir in ["..", "../src"]:
    if not lib_dir in sys.path:
        sys.path.append(lib_dir)
from lm_utils import *

In [10]:
language_model = make_lm(train_df.HUMAN_TRANSCRIPTION.sum())

In [11]:
from tqdm.auto import tqdm
from datastruct import *
from common import *
from space_fixer import SpaceFixer

# for ht_line, mt_line in tqdm(train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].values[7:]):
#     ht_words = word_regex.split(ht_line)
#     mt_words = word_regex.split(mt_line)
#     #words_ = [remove_cap(word) for word in words]
#     #vocab.add_sentence(words)
#     dmatrix = build_path_matrix(mt_words, vocabs)
#     finished_paths = extract_paths(dmatrix)
#     for k in resplit_paths(finished_paths, mt_words):
#         variant = " ".join(k)
#         print(variant)
#     break
CUTOFF = 3

In [12]:
def extract_spaces_dict(x):
    result = dict()
    last_index = 0
    for w in word_regex2.split(x):
        if not word_regex.match(w):
            last_index += 1
        else:
            result[last_index] = w
    if not last_index in result:
        result[last_index] = ""
    return result
    

In [13]:

train_df["SYSTEM_TRANSCRIPTION_raw"] = train_df.SYSTEM_TRANSCRIPTION
train_df["SYSTEM_TRANSCRIPTION"] = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

ht_sequences_train = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_sequences_train = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_train = train_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values

mt_texts_train = train_df.SYSTEM_TRANSCRIPTION.values
fixer = SpaceFixer(MAX_WORDS)
fixer.fill(ht_sequences_train)

  0%|          | 0/1875 [00:00<?, ?it/s]

In [15]:
from common import correct_sigmas_in_word

def join_if_tuple(s):
    if isinstance(s, str):
        return s
    return " ".join([correct_sigmas_in_word(x) for x in s])

def postprocess_spaces(spaces_after):
    if len(spaces_after) < 2:
        return spaces_after
    if spaces_after[-1]!=".":
        spaces_after[-1] = ''
    return spaces_after

corrected_texts = []
# mt_sequences_train = mt_sequences_train[-1:]
# mt_spaces_train = mt_spaces_train[-1:]
# mt_texts_train = mt_texts_train[-1:]
# ht_sequences_train = ht_sequences_train[-1:]
corrected_count = 0

train_iter = zip(mt_sequences_train, mt_spaces_train, mt_texts_train, ht_sequences_train)
for i, (mt_words, line_spaces, mt_orig, ht_orig) in enumerate(tqdm(train_iter, total=len(mt_sequences_train))):
    # print(line_spaces)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    best = mt_orig
    # temporary disable the following lines
    replacements = [
        [join_if_tuple(w) + s for w, s in zip(mt_split, postprocess_spaces(spaces_after))]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=CUTOFF)
    ]
    replacements = ["".join(words) for words in replacements]

    #replacements = [
    #    ("".join([w + s for w, s in zip(mt_split, spaces_after)]), spaces_after) 
    #    
    #]
    # replacements, spaces_after = list(zip(*replacements))
    # replacements = list(replacements)
    if len(replacements) > 2:
        print(i, len(replacements))
        print(replacements)
        # print(spaces_after)
        print(line_spaces)
        # break
    
    best, is_corrected = lm_score(mt_orig, replacements, lm=language_model, return_corrected=True)
    if is_corrected:
        corrected_count += 1
    corrected_texts.append(best)
    # break

  0%|          | 0/1875 [00:00<?, ?it/s]

In [16]:
# for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=5):
#     print(mt_split, refs, spaces_after)
# print(ht_orig)

In [17]:
unicodedata.name("σ"), unicodedata.name("ς")
corrected_count
# fixer.split_matrix
# best, is_corrected = lm_score(mt_orig, replacements, lm=language_model, return_corrected=True)
# best, ht_orig

553

In [18]:
# K = 6
# train_iter = zip(
#     mt_sequences_train[K:],
#     mt_spaces_train[K:],
#     mt_texts_train[K:],
#     ht_sequences_train[K:]
# )
# for i, (mt_words, line_spaces, mt_orig, ht_orig) in enumerate(tqdm(train_iter, total=len(mt_sequences_train[K:]))):
#     break
# print(mt_words, ht_orig)
# for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=0):
#     print(mt_split)
# #mt_words, ht_orig


In [19]:
# for mt_split, refs, spaces_after in fixer.resplit(mt_words, line_spaces):
#     print(mt_split, spaces_after, line_spaces)

# ht_orig, mt_orig

    

# [postprocess_sigmas(t) for t in corrected_texts]

In [20]:
# print(ht_sequences_train[-1])
#print("mt:", mt_words)
#for k, v, spaces_after in fixer.split_words(mt_words, line_spaces):
#    print(k, v, spaces_after)

In [21]:
# fixer.dmatrix
#fixer.split_matrix

In [22]:
from metrics import compute_metrics

In [23]:
ht_texts = train_df.HUMAN_TRANSCRIPTION.values
mt_texts = train_df.SYSTEM_TRANSCRIPTION_raw.values
ct = [lmr(t, lm=language_model) for t in corrected_texts]
# ct2 = [postprocess_sigmas(t) for t in ct]
ct2 = [lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model) for t in ct]
print("Unmodified:")
cerr_values_ht, cerr_values = compute_metrics(ht_texts, mt_texts, corrected_texts)
print("Corrected sigmas:")
cerr_values_ht1, cerr_values1 = compute_metrics(ht_texts, mt_texts, ct)
#print("corrected deltas:")
#cerr_values_ht2, cerr_values2 = compute_metrics(ht_texts, mt_texts, ct2)

Unmodified:
Candidate CER: 33.300649498390754
Candidate CERR: 0.9544254789018402
Corrected sigmas:
Candidate CER: 33.300649498390754
Candidate CERR: 0.9544254789018402


In [24]:
diff_cer = (cerr_values_ht - cerr_values)
idx = np.argsort(diff_cer)
for i in idx[:5]:
    print(diff_cer[i])
    print(repr(ct[i]))
    print(repr(ht_texts[i]), repr(mt_texts[i]))

-5.555555555555557
'τοῦ δυσιν εσαιλόγος σξωην ἐκρ'
'τοῦ Θῦ γίνεται Λόγος, ὡς ζωή νεκροῖς' 'τοῦ δυσιν εσαιλόγος ς ξωην ἐκρ'
-4.761904761904763
'ουτων πα δι ων δεξησιου'
'ουτων παιδιων δεξηται' 'ουτων πα διων δεξησιου'
-3.703703703703706
'δὲ α ἀποκριθεις αὐτῷ λεγει'
'ὁ δὲ ἀποκριθεὶς αὐτῷ λέγει·' 'δὲ αἀποκριθεις αὐτῷ λεγει'
-3.5714285714285694
'μωτς πατρῷον ἵκδὶ τ᾽ ρ οὐκ όν, τα'
'μήτε πατρῷον ἵκοιτ᾽ ἐς οἶκον' 'μωτς πατρῷον ἵκδὶ τ᾽ ρ οὐκόν, τα'
-3.030303030303024
'θὴ ποι ειται την η αθαίρεσιν, κι'
' Θῦ, ποιεῖται τήν καθαίρεσιν. Καί' 'θὴ ποι ειται την ηαθαίρεσιν, κι'


In [24]:
test_df["SYSTEM_TRANSCRIPTION_raw"] = test_df.SYSTEM_TRANSCRIPTION
test_df["SYSTEM_TRANSCRIPTION"] = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

In [25]:
mt_orig_test = test_df.SYSTEM_TRANSCRIPTION.values
mt_sequences_test = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_test = test_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values
corrected_texts_test = []
for i, (mt_words, line_spaces, mt_orig) in enumerate(tqdm(zip(mt_sequences_test, mt_spaces_test, mt_orig_test), total=len(mt_sequences_test))):
    # print(i)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    # temporary disable the following lines
    best = mt_orig
    # replacements = ["".join([w+s for w, s in zip(mt_split, spaces_after)]) for mt_split, refs, spaces_after in fixer.split_words(mt_words, mt_spaces)]
    replacements = [
        [join_if_tuple(w) + s for w, s in zip(mt_split, postprocess_spaces(spaces_after))]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=CUTOFF)
    ]
    replacements = ["".join(words) for words in replacements]
    best = lm_score(mt_orig, replacements, lm=language_model)
    corrected_texts_test.append(best)

  0%|          | 0/338 [00:00<?, ?it/s]

In [26]:
corrected_texts_test1 = [lmr(t, lm=language_model) for t in corrected_texts_test]

# corrected_texts_test2 = [
#     lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model)
#     for t in corrected_texts_test1
# ]
# corrected_texts_test2 = [postprocess_sigmas(t) for t in corrected_texts_test1]

In [27]:
submission = pd.DataFrame(
    zip(test_df.IMAGE_PATH, corrected_texts_test1),
    columns=["ImageID", "Transcriptions"]
)
submission.sample()

,ImageID,Transcriptions
110,105 Bodleian-Library-MS-Barocci-59_00085_fol-4...,τὲ πρμον ην τὴν αν θησιν τὴν σρ


In [28]:
submission.to_csv("submission.csv", index=False)

In [29]:
%aicrowd submission create -c htrec-2022 -f submission.csv

submission.csv ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 45.3/43.6 KB • 428.2 kB/s • 0:00:00

╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯

                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/htrec-2022/submissions/191847              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/htrec-2022/submissions?my_submissions=true │
│                  │                                                                               │
│      Leaderboard │ https://www.aicrowd.com/challenges/htrec-2022/leaderboards                    │
│                  │                                                                               │
│ Discussion forum │ https://discourse.aicrowd.com/c/htrec-2022                                    │
│                  │                                                                               │
│   Challenge page │ https://www.aicrowd.com/challenges/htrec-2022                                 │
└──────────────────┴───────────────────────────────────────────────────────────────────────────────┘

{'submission_id': 191847, 'created_at': '2022-07-01T11:46:48.870Z'}


In [ ]:
#test_spaces = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces1 = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces2 = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values

In [ ]:
#from collections import Counter
#Counter([x for xs in test_spaces for xss in xs for x in xss])

In [ ]:
#Counter([x for xs in train_spaces1 for xss in xs for x in xss])

In [ ]:
# Counter([x for xs in train_spaces2 for xss in xs for x in xss])